# LESSON 7: Image Negation (Inverse)
## Biomedical Image Processing Techniques

In this lesson:
- What is image negation
- Mathematical formula
- Applications in medical imaging
- Comparison with original

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. What is Image Negation?

**Image negation** (or inverse) reverses the intensity values.

### Formula:
$$g(x,y) = (L-1) - f(x,y)$$

For 8-bit images ($L = 256$):
$$g(x,y) = 255 - f(x,y)$$

**Effect:**
- Black (0) becomes White (255)
- White (255) becomes Black (0)
- Like a photographic negative

In [ ]:
def image_negation(image):
    """
    Compute the negative (inverse) of an image.
    
    Parameters:
    - image: input grayscale image (uint8)
    
    Returns:
    - negative: inverted image (uint8)
    """
    return 255 - image

In [ ]:
# Visualize the transformation function
x = np.arange(0, 256)
y = 255 - x

plt.figure(figsize=(8, 6))
plt.plot(x, y, 'b-', linewidth=2, label='Negation: g = 255 - f')
plt.plot(x, x, 'k--', alpha=0.5, label='Identity: g = f')
plt.xlabel('Input Pixel Value (f)', fontsize=12)
plt.ylabel('Output Pixel Value (g)', fontsize=12)
plt.title('Image Negation Transformation', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])
plt.gca().set_aspect('equal')
plt.show()

## 2. Simple Example

In [ ]:
# Create a simple test image
np.random.seed(42)
test_image = np.random.randint(0, 256, (200, 200), dtype=np.uint8)

# Add some patterns
test_image[50:150, 50:150] = 200  # Bright square
test_image[80:120, 80:120] = 50   # Dark square inside

# Apply negation
negative = image_negation(test_image)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original Image')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(negative, cmap='gray', vmin=0, vmax=255)
plt.title('Negative Image\ng = 255 - f')
plt.colorbar()

plt.suptitle('Image Negation Example', fontsize=14)
plt.tight_layout()
plt.show()

# Show some pixel values
print("Sample pixel values:")
print(f"Original bright region: {test_image[100, 100]}")
print(f"Negative bright region: {negative[100, 100]}")
print(f"Sum: {test_image[100, 100] + negative[100, 100]}")

## 3. Gradient Example

In [ ]:
# Create a gradient image
gradient = np.tile(np.linspace(0, 255, 256), (100, 1)).astype(np.uint8)
gradient_neg = image_negation(gradient)

plt.figure(figsize=(14, 6))

plt.subplot(2, 2, 1)
plt.imshow(gradient, cmap='gray', vmin=0, vmax=255)
plt.title('Original Gradient')
plt.colorbar()
plt.ylabel('Black -> White')

plt.subplot(2, 2, 2)
plt.imshow(gradient_neg, cmap='gray', vmin=0, vmax=255)
plt.title('Negative Gradient')
plt.colorbar()
plt.ylabel('White -> Black')

plt.subplot(2, 2, 3)
plt.plot(gradient[50, :], 'b-', linewidth=2, label='Original')
plt.xlabel('Pixel Position')
plt.ylabel('Intensity')
plt.title('Original Profile')
plt.ylim([0, 260])
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.plot(gradient_neg[50, :], 'r-', linewidth=2, label='Negative')
plt.xlabel('Pixel Position')
plt.ylabel('Intensity')
plt.title('Negative Profile')
plt.ylim([0, 260])
plt.grid(True, alpha=0.3)

plt.suptitle('Gradient and Its Negative', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Histogram Analysis

In [ ]:
# Analyze histogram changes
np.random.seed(123)
sample = np.random.randint(50, 180, (200, 200), dtype=np.uint8)
sample_neg = image_negation(sample)

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.imshow(sample, cmap='gray', vmin=0, vmax=255)
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(sample_neg, cmap='gray', vmin=0, vmax=255)
plt.title('Negative')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.hist(sample.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Original', color='blue')
plt.hist(sample_neg.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Negative', color='red')
plt.xlabel('Intensity')
plt.ylabel('Count')
plt.title('Histogram Comparison')
plt.legend()
plt.xlim([0, 255])

plt.suptitle('Histogram is Mirrored in Negation', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Medical Application: Mammography Enhancement

In mammography:
- Original: Dense tissue appears **white**, fat appears **dark**
- Negative: Dense tissue appears **dark**, fat appears **white**

Some radiologists prefer the negative view for detecting certain abnormalities.

In [ ]:
# Simulate a mammography-like image
np.random.seed(789)

# Base tissue (fatty tissue - darker)
mammo = np.random.randint(30, 80, (256, 256), dtype=np.uint8)

# Add dense tissue regions (brighter)
from scipy.ndimage import gaussian_filter

# Breast outline (elliptical shape)
y, x = np.ogrid[:256, :256]
breast_mask = ((x - 128)**2 / 100**2 + (y - 128)**2 / 120**2) <= 1
mammo[~breast_mask] = 0

# Dense tissue areas
mammo[100:180, 80:150] += np.random.randint(40, 80, (80, 70), dtype=np.uint8)
mammo[60:100, 100:140] += np.random.randint(30, 60, (40, 40), dtype=np.uint8)

# Add a suspicious lesion (small bright spot)
lesion_y, lesion_x = 130, 120
lesion_mask = (x - lesion_x)**2 + (y - lesion_y)**2 <= 8**2
mammo[lesion_mask] = 220

# Smooth the image
mammo = gaussian_filter(mammo, sigma=2).astype(np.uint8)
mammo[~breast_mask] = 0

# Create negative
mammo_neg = image_negation(mammo)
mammo_neg[~breast_mask] = 0  # Keep background black

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(mammo, cmap='gray')
plt.title('Original Mammogram\n(Dense tissue = white)')
plt.axis('off')
# Mark the lesion
circle1 = plt.Circle((lesion_x, lesion_y), 15, fill=False, color='red', linewidth=2)
plt.gca().add_patch(circle1)
plt.text(lesion_x + 20, lesion_y, 'Lesion', color='red', fontsize=10)

plt.subplot(1, 2, 2)
plt.imshow(mammo_neg, cmap='gray')
plt.title('Negative View\n(Dense tissue = dark)')
plt.axis('off')
# Mark the lesion
circle2 = plt.Circle((lesion_x, lesion_y), 15, fill=False, color='yellow', linewidth=2)
plt.gca().add_patch(circle2)
plt.text(lesion_x + 20, lesion_y, 'Lesion', color='yellow', fontsize=10)

plt.suptitle('Mammography: Original vs Negative View', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Medical Application: Digital Subtraction Angiography (DSA)

In angiography, negation helps visualize blood vessels:
- X-ray: vessels with contrast agent appear **white**
- Negative: vessels appear **dark** (easier to see against bright background)

In [ ]:
# Simulate an angiography image
np.random.seed(321)

# Background (tissue)
angio = np.random.randint(80, 120, (256, 256), dtype=np.uint8)

# Add blood vessels (bright lines representing contrast agent)
def draw_vessel(img, x1, y1, x2, y2, width, intensity):
    """Draw a vessel as a line with given width."""
    length = int(np.sqrt((x2-x1)**2 + (y2-y1)**2))
    for i in range(length):
        t = i / length
        cx = int(x1 + t * (x2 - x1))
        cy = int(y1 + t * (y2 - y1))
        y, x = np.ogrid[:256, :256]
        mask = (x - cx)**2 + (y - cy)**2 <= width**2
        img[mask] = np.maximum(img[mask], intensity)

# Main vessel
draw_vessel(angio, 128, 0, 128, 256, 6, 220)

# Branches
draw_vessel(angio, 128, 60, 60, 100, 4, 210)
draw_vessel(angio, 128, 60, 200, 90, 4, 210)
draw_vessel(angio, 128, 150, 70, 200, 3, 200)
draw_vessel(angio, 128, 150, 190, 180, 3, 200)
draw_vessel(angio, 70, 200, 40, 240, 2, 190)
draw_vessel(angio, 190, 180, 220, 220, 2, 190)

# Smooth
angio = gaussian_filter(angio, sigma=1).astype(np.uint8)

# Negative
angio_neg = image_negation(angio)

In [ ]:
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(angio, cmap='gray')
plt.title('Original Angiogram\n(Vessels = bright/white)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(angio_neg, cmap='gray')
plt.title('Negative Angiogram\n(Vessels = dark/black)')
plt.axis('off')

plt.suptitle('Angiography: Original vs Negative', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Property: Double Negation

In [ ]:
# Double negation returns the original
original = test_image.copy()
first_neg = image_negation(original)
double_neg = image_negation(first_neg)

plt.figure(figsize=(15, 4))

plt.subplot(1, 3, 1)
plt.imshow(original, cmap='gray', vmin=0, vmax=255)
plt.title('Original (f)')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(first_neg, cmap='gray', vmin=0, vmax=255)
plt.title('First Negation (255-f)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(double_neg, cmap='gray', vmin=0, vmax=255)
plt.title('Double Negation\n255-(255-f) = f')
plt.axis('off')

plt.suptitle('Double Negation Returns Original Image', fontsize=14)
plt.tight_layout()
plt.show()

# Verify mathematically
print(f"Original == Double Negative: {np.array_equal(original, double_neg)}")

## Summary

What we learned:

1. **Image Negation**: $g = (L-1) - f = 255 - f$

2. **Effect**: Inverts all intensity values
   - Black becomes white
   - White becomes black

3. **Histogram**: Gets mirrored around center

4. **Medical Applications**:
   - Mammography viewing
   - Angiography visualization
   - Personal preference for radiologists

5. **Property**: Double negation = original image